In [3]:
#!/usr/bin/env python
from scipy.signal import hilbert
from scipy.signal import welch
from scipy import signal
import pandas as pd
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import os
import neurokit2 as nk
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import glob
from joblib import Parallel, delayed


In [4]:
# call:  python features_DFA.py -data_dir EPOCHS -output_dir RESULTS -part_info EPOCHS/participants.txt -lfrequ 8 -hfrequ 14

def get_channel_hurst(ch_data,sfreq):

    scale = nk.expspace(1*sfreq, 20*sfreq, 40, base=2).astype(np.int64)

    analytic_signal = hilbert(ch_data)
    amplitude_envelope = np.abs(analytic_signal)

    try:
        hurst_fh, _ = nk.fractal_hurst(amplitude_envelope, scale=scale, show=False)
    except:
        hurst_fh = float('nan')

    try:
        hurst_dfa, _ = nk.fractal_dfa(amplitude_envelope, scale=scale, show=False)
    except:
        hurst_dfa = float('nan')

    return  hurst_fh, hurst_dfa


In [6]:
# Define input and output directories
out_dir = 'data/output/DFA/'
in_dir = 'data/input/continuous/'
lfreq = 0.1  # Low frequency filter
hfreq = 40   # High frequency filter

# Ensure the output directory exists
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Find all .fif files
paths = glob.glob(in_dir + '*.fif')
paths.sort()
paths = paths[2:4]  # Adjust file selection if needed

# Lists to store results
HURST_FH = []
HURST_DFA = []
sub_ids = []
days = []
conditions = []

# Extract metadata from filenames
def extract_metadata(path):
    filename = os.path.basename(path)  # Extract filename
    parts = filename.split('-')  # Assuming a naming convention like "subX-dayY-conditionZ.fif"
    sub = parts[0]  # Example: "sub0"
    day = parts[1]  # Example: "day2"
    condition = parts[2].split('.')[0]  # Example: "jhana" (removing ".fif")
    return sub, day, condition

# Loop over each file
for path in paths:
    print(f"Analyzing DFA of {path}")

    # Extract metadata
    sub, day, condition = extract_metadata(path)
    sub_ids.append(sub)
    days.append(day)
    conditions.append(condition)

    # Load data
    raw = mne.io.read_raw_fif(path, preload=True)
    raw.interpolate_bads(reset_bads=True)
    data = raw.get_data()[0:32, :]  # Use only first 32 channels
    fs = 256  # Sampling frequency
    sig_length = min(data.shape[1] / fs, 200)  # Ensure max 200s of data
    nr_channels = data.shape[0]  # Number of channels

    # Cut data
    cut = int(sig_length * fs)
    data = data[:, :cut]

    # Apply filtering
    data_filt = mne.filter.filter_data(data, sfreq=fs, l_freq=lfreq, h_freq=hfreq, verbose=False)

    # Run multiprocessing using joblib
    results = Parallel(n_jobs=-1)(delayed(get_channel_hurst)(data_filt[ch, :], fs) for ch in range(nr_channels))

    # Store results
    HURST_FH.append(np.mean(pd.DataFrame(results)[0]))
    HURST_DFA.append(np.mean(pd.DataFrame(results)[1]))

# Save results with metadata
output_df = pd.DataFrame({
    'sub': sub_ids,
    'day': days,
    'condition': conditions,
    'HURST_FH': HURST_FH,
    'HURST_DFA': HURST_DFA
})

# Save to CSV
output_df.to_csv(f'{out_dir}/DFA_{lfreq}_{hfreq}.csv', index=False, sep=',')

print("Processing complete. Results saved.")


Analyzing DFA of data/input/continuous/sub0-day2-jhana-raw.fif
Opening raw data file data/input/continuous/sub0-day2-jhana-raw.fif...
    Range : 512 ... 323799 =      2.000 ...  1264.840 secs
Ready.
Reading 0 ... 323287  =      0.000 ...  1262.840 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
Analyzing DFA of data/input/continuous/sub0-day2-mindfulness-raw.fif
Opening raw data file data/input/continuous/sub0-day2-mindfulness-raw.fif...
    Range : 512 ... 294535 =      2.000 ...  1150.527 secs
Ready.
Reading 0 ... 294023  =      0.000 ...  1148.527 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 23 sensor positions
Interpolating 9 sensors
Processing complete. Results saved.
